
## Loading libraries and modules

In [1]:
%pylab qt5

Populating the interactive namespace from numpy and matplotlib


In [2]:
mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = ':'
mpl.rcParams['grid.linewidth'] = 0.5

In [3]:
import drivers
#reload(drivers.Agilent_EXA)

In [4]:
from importlib import reload
import lib2
from lib2.Measurement import *
from lib2.SingleToneSpectroscopy import *
from lib2 import MeasurementResult
from lib2.FourWaveMixingBase import *
import csv

In [24]:
reload(lib2.SingleToneSpectroscopy)

<module 'lib2.SingleToneSpectroscopy' from 'C:\\Users\\PAINMASTER\\Measurement\\lib2\\SingleToneSpectroscopy.py'>

# Single-Tone Spectroscopy


In [26]:
STS = SingleToneSpectroscopy("I-Single_Tone_Spectroscopy", "Xmons_Nb_bandage", vna = 'vna1', src = 'yok3')
vna_parameters = {"bandwidth":1000, "freq_limits":(7.5e9, 8.2e9), "nop":5001, "power":-50, "averages":1}
STS.set_fixed_parameters(vna_parameters)
STS.set_swept_parameters({'current': (STS._src.set_current, linspace(2e-3, 4.5e-3, 251))})
result = STS.launch()
result.visualize()

vna1 is already initialized
yok3 is already initialized
Started at:  2017-08-03 21:59:47.904056


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s, [current: 4.50e-03]: , average cycle time: 5.37 s          

(<matplotlib.figure.Figure at 0x10e94208>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x00000000153BDBE0>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x0000000015979630>], dtype=object),
 (<matplotlib.axes._axes.Axes at 0x159efeb8>,
  <matplotlib.axes._axes.Axes at 0x15a44b70>))

In [28]:
result.save()

In [6]:
STS._actual_devices['yok3'].set_appropriate_range(10e-3)
STS._actual_devices['yok3'].set_current_limits(-12e-3,12e-3)

In [ ]:
vna2.del_all_meas()

In [ ]:
vna2.set_power(-45)
vna2.set_nop(1001)
vna2.set_bandwidth(300)
vna2.prepare_for_stb()
vna2.sweep_single()
vna2.wait_for_stb()
data = vna2.get_tracedata('AmpPha')[0]
plt.plot(vna2.get_freqpoints(), 20*np.log10(data))

In [17]:
path = result.get_save_path()
I = result.get_data()["current"]
freq = result.get_data()["frequency"]
with open( path + "current.csv", "w") as file:
    writer = csv.writer(file )
    writer.writerow( I )

with open( path + "frequency.csv", "w") as file:
    writer = csv.writer( file )
    writer.writerow( freq )

with open( path + "data_real.csv", "w") as file:
    writer = csv.writer( file )
    for data_row in result.get_data()["data"]:
        writer.writerow( real(data_row) )
        
with open( path + "data_imag.csv", "w") as file:
    writer = csv.writer( file )
    for data_row in result.get_data()["data"]:
        writer.writerow( imag(data_row) )
result.save()

_______________

## Four Wave mixing Continuous

In [ ]:
reload(drivers.E8257D);

In [ ]:
#reload(lib2.Measurement)
reload(lib2.FourWaveMixingBase)
from lib2.FourWaveMixingBase import *

In [ ]:
WMM = FourWaveMixingBase('I-Measurement_with_2_gens', 'SPS_old', 60, exa = 'exa', src_plus = 'mxg', src_minus = 'exg')

In [ ]:
exa_pars = {"bandwidth":5, "nop":1001, "centerfreq":4.558e9, "span":200e3, "averages":20, "avg_status":1}
WMM.set_fixed_parameters(exa_pars,delta = 5e3, power_minus=-15)

In [ ]:
powers = linspace(-29,9,161)
WMM.set_swept_parameters(WMM.powers_plus_setter,powers)

In [ ]:
result = WMM.launch()

________________

In [ ]:
WMM._sources_off()

In [ ]:
WMM._actual_devices["exa"].

In [ ]:
WMM.close_devs("mxg")

_________________

### Result actions:

In [ ]:
last_data = result.get_data()

In [ ]:
plt.plot(result._prepare_data_for_plot(last_data)[3])
plt.show()

In [ ]:
result.get_context().get_equipment()

In [ ]:
D = result.get_data()
D

In [31]:
result.visualize()

(<matplotlib.figure.Figure at 0x13db04e0>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x000000000C131668>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x0000000013E874E0>], dtype=object),
 (<matplotlib.axes._axes.Axes at 0x10853ef0>,
  <matplotlib.axes._axes.Axes at 0xd608d30>))


______________

In [ ]:
setup()

## Rabi oscillations IQ mixer

In [1]:
import drivers
from importlib import reload
from lib import data_management as dm

In [8]:
reload(drivers.KeysightAWG)

<module 'drivers.KeysightAWG' from 'C:\\Users\\PAINMASTER\\Measurement\\drivers\\KeysightAWG.py'>

In [9]:
from drivers.KeysightAWG import *

In [10]:
from drivers import *

In [11]:
exa = Agilent_EXA.Agilent_EXA_N9010A("EXA")

In [12]:
if_freq = 200e6
q_freq = 4.558e9
calibrations = (dm.load_IQMX_calibration_database("SPS_mixer1", -6))
cal = calibrations.get(frozenset(dict(lo_power=15, ssb_power=-40, lo_frequency=q_freq+if_freq, 
                                      if_frequency=if_freq, waveform_resolution=0.1).items()))
pb = PulseBuilder(cal)

In [14]:
awg = KeysightAWG.KeysightAWG("AWG2")

In [15]:
cal.get_optimization_results()

({'dc_offsets': array([-0.00572362, -0.0149987 ]),
  'dc_offsets_open': None,
  'if_amplitudes': array([-1.12361941,  1.08115163]),
  'if_offsets': array([-0.00572362, -0.01736287]),
  'if_phase': array([ 1.70583871])},
 {'dc': -102.0552978515625,
  'if': [-39.952842712402344, -73.05088806152344, -102.57215118408203]})

In [16]:
pulse_seq = pb.add_sine_pulse(1.2, 0).add_zero_pulse(48.8).build()
awg.output_pulse_sequence(pulse_seq)

In [17]:
pulse_seq.plot()

In [18]:
len(pulse_seq.get_I_waveform())

501

In [19]:
round(1.2/0.1)

12

In [20]:
from time import sleep

In [21]:
exa.setup_swept_sa(center_freq=q_freq,span=100e6,nop=1001,rbw=10e3)

In [23]:
plt.ion()
plt.clf()
exa.setup_swept_sa(center_freq=q_freq,span=10e3,nop=2001,rbw=3)
exa.set_averages(3)
exa.set_average(1)
for dur in linspace(5,46,201):
    pulse_seq = pb.add_sine_pulse(dur, 0).add_zero_pulse(100-dur).build()
    awg.output_pulse_sequence(pulse_seq)
    #sleep(1)
    intens = exa.make_sweep_get_data()
    print('\r'+str(max(intens)), flush=True,end='')
    plot([dur], [intens[1000]], "go")
    plt.show()
    plt.pause(0.1)

-130.395462036

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


-130.996185303

KeyboardInterrupt: 

In [ ]:
intens

In [ ]:
exa.setup_swept_sa(q_freq, 10e3,1001,10)

______________